In [4]:
%cd '/mnt/c/Users/malin/Documents/Facultate/honours/UMCG/ICU_Augment_and_Detect'

/mnt/c/Users/malin/Documents/Facultate/honours/UMCG/ICU_Augment_and_Detect


In [5]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
from keras.layers.normalization import BatchNormalization
from augment.face_org import * 
import random

Using TensorFlow backend.


In [7]:
image_folder_sick = '/mnt/c/Users/malin/Documents/Facultate/honours/UMCG/ICU_Augment_and_Detect/data/parsed/sick'
image_folder_healthy = '/mnt/c/Users/malin/Documents/Facultate/honours/UMCG/ICU_Augment_and_Detect/data/parsed/healthy'
image_folder_altered = '/mnt/c/Users/malin/Documents/Facultate/honours/UMCG/ICU_Augment_and_Detect/data/parsed/altered'
image_folder_cfd = '/mnt/c/Users/malin/Documents/Facultate/honours/UMCG/ICU_Augment_and_Detect/data/parsed/cfd'
image_size = 217

In [8]:

def load_data(folder_sick, folder_healthy, image_size, type):
    files_healthy = os.listdir(folder_healthy)
    files_sick = os.listdir(folder_sick)
    data = []
    labels = []

    for filename in files_healthy:
        sick = 0
        full_path = folder_healthy + "/" + str(filename)
        if type in filename and os.path.isfile(full_path):
            image =  cv2.imread(full_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, dsize=(image_size, image_size), interpolation=cv2.INTER_CUBIC)
            data.append(np.asarray(image, dtype = np.int32))
            labels.append(np.asarray(sick, dtype = np.int32))
    for filename in files_sick:
        sick = 1
        full_path = folder_sick + "/" + str(filename)
        if type in filename and os.path.isfile(full_path):
            image =  cv2.imread(full_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, dsize=(image_size, image_size), interpolation=cv2.INTER_CUBIC)
            data.append(np.asarray(image, dtype = np.int32))
            labels.append(np.asarray(sick, dtype = np.int32))

    events = list(zip(data, labels))
    random.shuffle(events)
    data, labels = zip(*events)
    # print(data)
    return np.asarray(data, dtype=np.int32), np.asarray(labels, dtype=np.int32)


In [9]:
key = "mouth"
test_images_mouth, test_labels_mouth = load_data(image_folder_sick, image_folder_healthy, image_size, key)
train_images_mouth, train_labels_mouth = load_data(image_folder_altered, image_folder_cfd, image_size, key)


In [11]:
model = models.Sequential()

model.add(layers.Conv2D(image_size, (3, 3), activation='relu', input_shape=(image_size, image_size, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(image_size, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(int(image_size/2), (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(int(image_size/2), (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(int(image_size/4), (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(int(image_size/4), (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(int(image_size/8), (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(int(image_size/8), (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(int(image_size/16), (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Conv2D(int(image_size/16), (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.AveragePooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(52, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='softmax'))


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 215, 215, 217)     6076      
_________________________________________________________________
batch_normalization (BatchNo (None, 215, 215, 217)     868       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 213, 213, 217)     424018    
_________________________________________________________________
batch_normalization_1 (Batch (None, 213, 213, 217)     868       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 106, 106, 217)     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 104, 104, 108)     211032    
_________________________________________________________________
batch_normalization_2 (Batch (None, 104, 104, 108)     4

In [13]:
model.compile(optimizer='adam',
              loss="binary_crossentropy",
              metrics=['accuracy'])

history = model.fit(train_images_mouth, train_labels_mouth, epochs=10, 
                    validation_data=(test_images_mouth, test_labels_mouth))

Train on 686 samples, validate on 74 samples
Epoch 1/10


### Evaluate the model

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.show()


In [ ]:
plt.title('Learning Curves')
plt.xlabel('Epoch')
plt.ylabel('Cross Entropy')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_images_mouth,  test_labels_mouth, verbose=2)
print(test_acc)

Our simple CNN has achieved a test accuracy of over 70%. Not bad for a few lines of code! For another CNN style, see an example using the Keras subclassing API and a `tf.GradientTape` [here](https://www.tensorflow.org/tutorials/quickstart/advanced).